# CODE FOR LAKE STABILITY AND BUOYANCY FREQUENCY

# <font color=red>Read me before running the code <font>

This jupyter file contains python code for calculating lake stability and safety margin. To runn consistently the provided codes should be better to consider the guidance and keypoints as mentioned below:


1.  This code requires three kinds of data which vary to depth. The first type is the one that contains the vertical profiles for temperature, pressure, and salinity; the second must contains CO2 and CH4; and the last data must contains area.


2.  Start by creating new folder which will be composed by other three sub-folders corresponding to the required data.


3.  All data files must be in __.xlsx__ format.


4.  After copying the data in each and evry sub-folder, make sure that the data are clean without having any missing entry and negative depth value.


5.  Check if the required columns are named as: 
    
    -  __<font color=orange>Depth (m)<font>__         
    -  __<font color=orange>Press_corr<font>__  
    -  __<font color=orange>Temp<font>__  
    -  __<font color=orange>Salinity (g/l)<font>__  
    -  __<font color=orange>CO2 concentration (mol/l)<font>__  
    -  __<font color=orange>CH4 concentration (mol/l)<font>__
    -  __<font color=orange>area km2<font>__
    
    __<font color=black>It is better to copy from here each name and positioning it in the data files.<font>__


6.  Within the code there is only cell that requires to change some code lines (__Cell 2__) where more information are given as comments in that cell.


7.  Run every cell that contains codes without jumping any code. Be informed that any line which is satarting with __# symbol__ stands for comment (don't remove the symbol). 

### Cell 1: For libraries, don't change anything within the cell

In [ ]:
import pandas as pd
import os
import glob
import numpy as np
import seawater as sw
from seawater.constants import OMEGA, earth_radius
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import warnings
warnings.filterwarnings('ignore')

### Cell 2: For requirements, make changes by reading the comment (#) lines for guidence 

In [ ]:
####################################################################################################################
#################### Change every line here except the ones for comments (#) #######################################
####################################################################################################################

#------ Path for the location where files for data that contains depth, temp, pressure, salinity,... are saved------

path_1 = '/home/modeste/Desktop/Test_code/Folder_1'

#------ Path for the location where one file for gas data:depth, CO2, CH4,... is saved------------------------------

path_2 = '/home/modeste/Desktop/Test_code/Folder_2/DT_3.xlsx'
 
#------ Path for the location where one file for area data:depth, area,... is saved---------------------------------

path_3 = '/home/modeste/Desktop/Test_code/Folder_3/area.xlsx'

#------ The depth increment in meter that you wish to consider for the mean ----------------------------------------

step = 3

#------ Latitude which should be cosnidered during buoyancy frequency computation ----------------------------------

lat = [-1.80126]

#------ the path for the location where the outputs shuold be saved ------------------------------------------------

path_4 = '/home/modeste/Desktop/Test_code'

##################################### END OF CHANGES ###############################################################

### Cell 4: Mean computation for the data  at certain depth

In [ ]:
####################################################################################################################
#################### continue without changing anything below, just run every cell with codes ######################
####################################################################################################################

os.mkdir(path_1+'/Mean_1')
os.mkdir(path_4+'/Output')

In [ ]:
def compute_mean(df, step, use_columns):
    max_depth = int(df['Depth (m)'].max())
    result = []
    if type (step) == float and step >= 0.5:
        for i in np.arange(step,max_depth+2*step, step):
            v = [i-step] + list(df[(df['Depth (m)'] >= i-step) & (df['Depth (m)'] < i)].mean())[1:]
            result.append(v)
    elif type (step) == int:
        for i in np.arange(step,max_depth+2*step, step):
            v = [i-step] + list(df[(df['Depth (m)'] >= i-step) & (df['Depth (m)'] < i)].mean())[1:]
            result.append(v)
    else:
        message = print("The step can't work")
    
    df = pd.DataFrame(result, columns= use_columns)
    
    return df

__For varied data__

In [ ]:
input_files = glob.glob(os.path.join(path_1, "*.xlsx"))
for iteration, file in enumerate(input_files):
    use_columns_3 = ['Depth (m)', 'Press_corr', 'Temp', ' Salinity (g/l)']
    DF_C = pd.read_excel(file, usecols = use_columns_3)
    results_3 = compute_mean(DF_C, step, use_columns_3)
    results_3.to_excel(f"{path_1}/Mean_1/File_{iteration}.xlsx", index = False)

__For gases__

In [ ]:
DF_A = pd.read_excel(path_2, index_col=False)
use_columns_1 = ['Depth (m)','CO2 concentration (mol/l)', 'CH4 concentration (mol/l)']
DF_1 = pd.DataFrame(DF_A, columns = use_columns_1) 
results_1 = compute_mean(DF_1, step, use_columns_1)

__For area__

In [ ]:
DF_B = pd.read_excel(path_3, index_col = False)
use_columns_2 = ['Depth (m)','area km2']
DF_2 = pd.DataFrame(DF_B, columns = use_columns_2)
results_2 = compute_mean(DF_2, step, use_columns_2)

### Cell 5: For merging the files with respect to the number of the files in the given period

In [ ]:
files = glob.glob(os.path.join(path_1+'/Mean_1', "*.xlsx"))
merged = pd.concat(map(pd.read_excel, files), ignore_index = True)
grouped_df = merged.groupby(['Depth (m)']).agg({"Press_corr": 'mean', "Temp": 'mean', " Salinity (g/l)": 'mean'})
combined = grouped_df.reset_index()
combined['CO2 concentration (mol/l)'] = results_1['CO2 concentration (mol/l)']
combined['CH4 concentration (mol/l)'] = results_1['CH4 concentration (mol/l)']
combined['area km2'] = results_2['area km2']

### Cell 6: For calculating densities at various depth levels

In [ ]:
def calculate_densities(combined):
    combined['Temperature'] = combined['Temp']
    combined['CO2 (mol/l)'] = combined['CO2 concentration (mol/l)']
    combined['CH4 (mol/l)'] = combined['CH4 concentration (mol/l)']
    pf = combined
    pf['T2'] = pf['Temperature']**2
    pf['T3'] = pf['Temperature']**3
    pf['T4'] = pf['Temperature']**4
    pf['T5'] = pf['Temperature']**5
    pf['T6'] = pf['Temperature']**6 
    pf['p(T)'] = (0.9998395+6.7914E-5*pf['Temperature']-9.0894E-6*pf['T2']+1.017E-7*pf['T3']-1.2846E-9*pf['T4']+1.1592E-11*pf['T5']-5.0125E-14*pf['T6'])*1000
    pf['p(T,S)'] = pf['p(T)']*(1+(0.00075*pf[' Salinity (g/l)']))
    pf['p(T,S,CO2)'] = pf['p(T)']*(1+0.00075*pf[' Salinity (g/l)']+0.000284*pf['CO2 (mol/l)']*44.0099)
    pf['p(T,S,CH4)'] = pf['p(T)']*(1+0.00075*pf[' Salinity (g/l)']-0.00125*pf['CH4 (mol/l)']*16.04)
    pf['p(T,S,CH4,CO2)'] = pf['p(T)']*(1+0.00075*pf[' Salinity (g/l)']-0.00125*pf['CH4 (mol/l)']*16.04+0.000284*pf['CO2 (mol/l)']*44.0099)
    return pf

In [ ]:
data = calculate_densities(combined)
data.to_excel(f"{path_4}/Output/Final_dataset.xlsx", index = False)

__Plot for vertical density profiles__

In [ ]:
#----- Generated plot in this cell will be displayed and saved simultaneously--------------------------


figure(figsize = (6, 6), dpi = 80)

plt.plot(data['p(T)'], data['Depth (m)'] ,'b', linewidth = 2.5, label = r"$\rho (T)$")
plt.plot(data['p(T,S)'], data['Depth (m)'], 'r', linewidth = 2.5, label = r"$\rho (T,S)$")
plt.plot(data['p(T,S,CH4)'], data['Depth (m)'], 'y',  linewidth = 2.5, label = r"$\rho (T,S,CH_4)$")
plt.plot(data['p(T,S,CO2)'], data['Depth (m)'], 'black',  linewidth = 2.5, label = r"$\rho (T,S,CO_2)$")
plt.plot(data['p(T,S,CH4,CO2)'], data['Depth (m)'], 'indigo',  linewidth = 2.5, label = r"$\rho (T,S,CH_4,CO_2)$")
plt.ylim(0, data['Depth (m)'].max() + 20)

plt.legend(loc = "upper right")
plt.gca().invert_yaxis()
plt.ylabel("Depth (m)", fontweight = 'bold')
plt.xlabel("Density ( kg/m^3)", fontweight = 'bold')
plt.grid()
plt.savefig(f"{path_4}/Output/Figure_1.png", dpi = 300, bbox_inches = 'tight')

__Mean density and its corresponding depth__

In [ ]:
Mean_dens = data['p(T,S,CH4,CO2)'].mean()
DT = data
DT['Tolerance'] = abs(DT['p(T,S,CH4,CO2)'] - Mean_dens)
for i in range(DT.shape[0]):
    if DT['Tolerance'][i] == DT['Tolerance'].min():
        Z_mean= DT['Depth (m)'][i]
print("Mean density is:", Mean_dens, "kg/m^3, and the corresponding mean depth is:", Z_mean, "m")

### Cell 7: Calculation of stability

In [ ]:
DATA = data
ss = 0
for i in range(DATA.shape[0]):
    ss = ss + (DATA['Depth (m)'][i] - Z_mean)*(DATA['p(T,S,CH4,CO2)'][i] - Mean_dens)*DATA['area km2'][i]
ss = (9.81/2730)*ss    
print("Calculated stability is:", ss, "J/m^2")

### Cell 8: Buoyancy frequency

In [ ]:
def N(bvfr2):
    """
    Buoyancy frequency is the frequency with which a parcel or particle of
    fluid displaced a small vertical distance from its equilibrium position in
    a stable environment will oscillate. It will oscillate in simple harmonic
    motion with an angular frequency defined by
    .. math:: N = \\left(\\frac{-g}{\\sigma_{\\theta}}
              \\frac{d\\sigma_{\\theta}}{dz}\\right)^{2}
    Parameters
    ----------
    n2 : array_like
         Brünt-Väisälä Frequency squared [s :sup:`-2`]
    Returns
    -------
    n : array_like
        Brünt-Väisälä Frequency not-squared [s :sup:`-1`]
    Examples
    --------
    >>> import numpy as np
    >>> import oceans.sw_extras.sw_extras as swe
    >>> s = np.array([[0, 0, 0], [15, 15, 15], [30, 30, 30],[35,35,35]])
    >>> t = np.repeat(15, s.size).reshape(s.shape)
    >>> p = [[0], [250], [500], [1000]]
    >>> lat = [30,32,35]
    >>> swe.N(sw.bfrq(s, t, p, lat)[0])
    array([[0.02124956, 0.02125302, 0.02125843],
           [0.02110919, 0.02111263, 0.02111801],
           [0.00860812, 0.00860952, 0.00861171]])
    References
    ----------
    A.E. Gill 1982. p.54  eqn 3.7.15 "Atmosphere-Ocean Dynamics"
    Academic Press: New York. ISBN: 0-12-283522-0
    Jackett, David R., Trevor J. Mcdougall, 1995: Minimal Adjustment of
    Hydrographic Profiles to Achieve Static Stability. J. Atmos. Oceanic
    Technol., 12, 381-389. doi: 10.1175/1520-0426(1995)012<0381:MAOHPT>2.0.CO;2
    """
    bvfr2 = np.asanyarray(bvfr2)
    return np.sqrt(np.abs(bvfr2)) * np.sign(bvfr2)

In [ ]:
s = data[' Salinity (g/l)'].values
t = data['Temp'].values
p = data['Press_corr'].values
N_BF = N(sw.bfrq(s, t, p, lat)[0])
data_BF = pd.concat([data, pd.DataFrame(N_BF, columns = ['N'])], axis = 1)
data_BF['N^2'] = data_BF['N']**2

__Plot for safety margin__

In [ ]:
#----- Generated plot in this cell will be displayed and saved simultaneously--------------------------

figure(figsize = (6, 6), dpi = 80)

plt.plot(data_BF['N^2'].values, data_BF['Depth (m)'].values, color = 'blue')
plt.gca().invert_yaxis()
plt.xlabel('$N^2$ (1/s^2)')
plt.ylabel('Depth (m)')
plt.grid()
plt.savefig(f"{path_4}/Output/Figure_2.png", dpi = 300, bbox_inches = 'tight')
#plt.savefig('/home/modeste/Rema/For_Graphs/plots/With_python/DT_2010_2015.png', dpi = 300, bbox_inches = 'tight')

#### <font color=red>If you wish to run the code again, please go and change the required code lines in cell 2.  Other wise the code will run wrongly.<font>